# Base de datos

Funcion para crear la base de datos en sqlite 3

In [1]:
import sqlite3

def init_db():
    conn = sqlite3.connect('hotel_reservas_basica.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS habitaciones (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        tipo_habitacion TEXT,
        camas INTEGER,
        camas_ocupadas INTEGER
    )
    ''')
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS reservas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        numero_identificacion INTEGER,
        nombre TEXT,
        id_habitacion INTEGER,
        check_in DATE,
        check_out DATE,
        estado TEXT,
        numero_contacto INTEGER,
        email TEXT
    )
    ''')
    
    # Insertar datos iniciales de las habitaciones (4 ordinarias y 2 compartidas)
    cursor.execute('SELECT COUNT(*) FROM habitaciones')
    if cursor.fetchone()[0] == 0:
        habitaciones_data = [
            (101 + i, 'Individual', 1, 0) for i in range(3)
        ] + [
            (201 + i, 'King', 1, 0) for i in range(3)
        ] + [
            (301 + i, 'Compartida', 4, 0) for i in range(3)
        ]
        cursor.executemany('INSERT INTO habitaciones (id, tipo_habitacion, camas, camas_ocupadas) VALUES (?, ?, ?, ?)', habitaciones_data)
    
    conn.commit()
    conn.close()

In [2]:
# Ejecutar la función para inicializar la base de datos básica
init_db()
# Comprobar el contenido de las tablas
conn = sqlite3.connect('hotel_reservas_basica.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM habitaciones where tipo_habitacion = "King"')
habitaciones = cursor.fetchall()
conn.close()

habitaciones

[(201, 'King', 1, 1), (202, 'King', 1, 0), (203, 'King', 1, 1)]

In [3]:
# Comprobar el contenido de las tablas
conn = sqlite3.connect('hotel_reservas_basica.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM reservas')
reservas = cursor.fetchall()
conn.close()

reservas

[(1,
  1001818160,
  'Kenny Dong Jie Zhu Ye',
  201,
  '2024-07-09',
  '2024-07-09',
  'confirmada',
  3024424366,
  'kennyzhuye15@gmail.com'),
 (2,
  1001818160,
  'kenny',
  301,
  '2024-07-09',
  '2024-07-09',
  'confirmada',
  2332323,
  'kekff@gmail.com'),
 (3, 1, 'cas', 301, '2024-07-09', '2024-07-09', 'confirmada', 1, 'ca'),
 (4, 121, 'dada', 301, '2024-07-09', '2024-07-10', 'confirmada', 21, 'dad'),
 (5,
  13213,
  'afafaf',
  103,
  '2024-07-09',
  '2024-07-09',
  'confirmada',
  2313,
  'fafafa'),
 (6,
  1001818160,
  'Kenny Dong Jie Zhu Ye',
  302,
  '2024-07-25',
  '2024-07-25',
  'confirmada',
  3024424366,
  'kennyzhuye15@gmail.com'),
 (7,
  44244,
  'rgegr',
  203,
  '2024-07-09',
  '2024-07-17',
  'confirmada',
  4242,
  'gege')]

In [4]:
import sqlite3

# Conectar a la base de datos
conn = sqlite3.connect('hotel_reservas_basica.db')
cursor = conn.cursor()

# Ejecutar el query para obtener habitaciones disponibles
cursor.execute('''
    SELECT id_habitacion FROM reservas 
    WHERE (? <= check_out AND ? >= check_in)
''', ('2024-07-09', '2024-07-10'))
reservas = cursor.fetchall()

# Cerrar la conexión
conn.close()

# Mostrar las habitaciones disponibles
for i in reservas:
    print(i)

habitaciones_disponibles = [
    habitacion for habitacion in habitaciones
    if (habitacion[0],) not in reservas
    ]
habitaciones_disponibles

(201,)
(301,)
(301,)
(301,)
(103,)
(203,)


[(202, 'King', 1, 0)]

In [5]:
import sqlite3

# Definir los parámetros de la consulta
tipo_habitacion = 'King'
check_in = '2024-07-12'
check_out = '2024-07-20'

# Conectar a la base de datos
conn = sqlite3.connect('hotel_reservas_basica.db')
cursor = conn.cursor()

# Ejecutar el query para obtener habitaciones disponibles
cursor.execute('''
    SELECT * FROM habitaciones 
    WHERE tipo_habitacion = ? AND id NOT IN (
        SELECT id_habitacion FROM reservas
        WHERE (check_in BETWEEN ? AND ?) OR (check_out BETWEEN ? AND ?) OR (? BETWEEN check_in AND check_out) OR (? BETWEEN check_in AND check_out)
    )
''', (tipo_habitacion, check_in, check_out, check_in, check_out, check_in, check_out))
habitaciones_disponibles = cursor.fetchall()

# Cerrar la conexión
conn.close()

# Mostrar las habitaciones disponibles
habitaciones_disponibles


[(201, 'King', 1, 1), (202, 'King', 1, 0)]

In [6]:
# Definir los parámetros de la consulta
tipo_habitacion = 'Compartida'
check_in = '2024-07-12'
check_out = '2024-07-12'

conn = sqlite3.connect('hotel_reservas_basica.db')
cursor = conn.cursor()

cursor.execute('''
        SELECT h.id, h.tipo_habitacion, h.camas, h.camas_ocupadas, 
               (SELECT COUNT(*) FROM reservas r 
                WHERE r.id_habitacion = h.id 
                  AND ((r.check_in <= ? AND r.check_out >= ?) 
                       OR (r.check_in >= ? AND r.check_out <= ?))) as camas_ocupadas_periodo 
        FROM habitaciones h 
        WHERE h.tipo_habitacion = ? 
        GROUP BY h.id, h.tipo_habitacion, h.camas, h.camas_ocupadas 
        HAVING h.camas - camas_ocupadas_periodo > 0
        ''', (check_out, check_in, check_in, check_out, tipo_habitacion))

habitaciones_disponibles = cursor.fetchall()

# Cerrar la conexión
conn.close()

# Mostrar las habitaciones disponibles
habitaciones_disponibles

[(301, 'Compartida', 4, 3, 0),
 (302, 'Compartida', 4, 1, 0),
 (303, 'Compartida', 4, 0, 0)]